# Twitter Sentiment Analysis

The original notebook was based on TF1. This notebook modifies the dependencies to TF2. 

In [3]:
# Download data
!pip install kaggle
!bash ../src/data/download_dataset.sh

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 58 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 118 kB 6.9 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 14.7 MB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=73812 sha256=55b36ce38dd003705116e0c0e068cbaeffb2b80e920f3a2ecc44f85cc7e8f0e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-s1rj5j_o/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
  Created wheel for python-slugify: filename=python_slugify-4.0.0-py2.py3-none-any.whl size=5733 sha256=3bee03712b7caafdd00c38b2b409c52a35b124ce09ba0e6e1fe6addc12f67688
  Stored in directory: /tmp/pip-ephem-wheel-cache-s1rj5j_o/wheels/5f/a9/4b/aa1ee60ccbd4389a2fe11c9b94adf3d9fa61cecda46c1187a9
Successfully built kaggle python-slugify
  Consider adding this directory to PAT

In [2]:
!pip install --upgrade pip
!pip install gensim --upgrade
!pip show gensim
!pip install keras --upgrade
!pip install pandas --upgrade


    100% |████████████████████████████████| 1.4MB 8.5MB/s 
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1
     |████████████████████████████████| 24.2 MB 2.8 MB/s 
     |████████████████████████████████| 70 kB 7.1 MB/s 
  Created wheel for smart-open: filename=smart_open-1.9.0-py3-none-any.whl size=72255 sha256=92cf5d283a6957436133a797d4bb4982e70db3e59a698e764b98957e15ae12ca
  Stored in directory: /root/.cache/pip/wheels/8e/9f/cd/dbf5c1362c59abb699a218c1151679033b8ccb5b6db559d512
Successfully built smart-open
  Attempting uninstall: smart-open
    Found existing installation: smart-open 1.7.1
    Uninstalling smart-open-1.7.1:
      Successfully uninstalled smart-open-1.7.1
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Name: gensim
Version: 3.8.1
Summary: Python framework for fast Vector Space Modelling
Home-page: http://r

In [3]:
import tensorflow as tf
import tf.keras

ModuleNotFoundError: No module named 'tf'

In [4]:
# DataFrame
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# TensorFLow
import tensorflow as tf

import tf.keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

ModuleNotFoundError: No module named 'tf'

In [5]:
nltk.download('stopwords')

NameError: name 'nltk' is not defined

### Settings

In [6]:
# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
TRAIN_SIZE = 0.8

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 8
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

# EXPORT
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

### Read Dataset

### Dataset details
* **target**: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
* **ids**: The id of the tweet ( 2087)
* **date**: the date of the tweet (Sat May 16 23:58:44 UTC 2009)
* **flag**: The query (lyx). If there is no query, then this value is NO_QUERY.
* **user**: the user that tweeted (robotickilldozr)
* **text**: the text of the tweet (Lyx is cool)

In [7]:
dataset_filename = os.listdir("../input")[0]
dataset_path = os.path.join("..","input",dataset_filename)
print("Open file:", dataset_path)
df = pd.read_csv(dataset_path, encoding =DATASET_ENCODING , names=DATASET_COLUMNS)

NameError: name 'os' is not defined

In [8]:
print("Dataset size:", len(df))

NameError: name 'df' is not defined

In [9]:
df.head(5)

NameError: name 'df' is not defined

### Map target label to String
* **0** -> **NEGATIVE**
* **2** -> **NEUTRAL**
* **4** -> **POSITIVE**

In [10]:
decode_map = {0: "NEGATIVE", 2: "NEUTRAL", 4: "POSITIVE"}
def decode_sentiment(label):
    return decode_map[int(label)]

In [11]:
%%time
df.target = df.target.apply(lambda x: decode_sentiment(x))

NameError: name 'df' is not defined

In [12]:
target_cnt = Counter(df.target)

plt.figure(figsize=(16,8))
plt.bar(target_cnt.keys(), target_cnt.values())
plt.title("Dataset labels distribuition")

NameError: name 'Counter' is not defined

### Pre-Process dataset

In [13]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

NameError: name 'stopwords' is not defined

In [14]:
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [15]:
%%time
df.text = df.text.apply(lambda x: preprocess(x))

NameError: name 'df' is not defined

### Split train and test

In [16]:
df_train, df_test = train_test_split(df, test_size=1-TRAIN_SIZE, random_state=42)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

NameError: name 'df' is not defined

### Word2Vec 

In [17]:
%%time
documents = [_text.split() for _text in df_train.text] 

NameError: name 'df_train' is not defined

In [18]:
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)

NameError: name 'gensim' is not defined

In [19]:
w2v_model.build_vocab(documents)

NameError: name 'w2v_model' is not defined

In [20]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

NameError: name 'w2v_model' is not defined

In [21]:
%%time
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

NameError: name 'w2v_model' is not defined

In [22]:
w2v_model.most_similar("love")

NameError: name 'w2v_model' is not defined

### Tokenize Text

In [23]:
%%time
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.text)

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

NameError: name 'Tokenizer' is not defined

In [24]:
%%time
x_train = pad_sequences(tokenizer.texts_to_sequences(df_train.text), maxlen=SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(df_test.text), maxlen=SEQUENCE_LENGTH)

NameError: name 'pad_sequences' is not defined

### Label Encoder 

In [25]:
labels = df_train.target.unique().tolist()
labels.append(NEUTRAL)
labels

NameError: name 'df_train' is not defined

In [26]:
encoder = LabelEncoder()
encoder.fit(df_train.target.tolist())

y_train = encoder.transform(df_train.target.tolist())
y_test = encoder.transform(df_test.target.tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train",y_train.shape)
print("y_test",y_test.shape)

NameError: name 'df_train' is not defined

In [27]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print()
print("x_test", x_test.shape)
print("y_test", y_test.shape)

NameError: name 'x_train' is not defined

In [28]:
y_train[:10]

NameError: name 'y_train' is not defined

### Embedding layer

In [29]:
embedding_matrix = np.zeros((vocab_size, W2V_SIZE))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

NameError: name 'np' is not defined

In [30]:
embedding_layer = Embedding(vocab_size, W2V_SIZE, weights=[embedding_matrix], input_length=SEQUENCE_LENGTH, trainable=False)

NameError: name 'Embedding' is not defined

### Build Model

In [31]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

NameError: name 'Sequential' is not defined

### Compile model

In [32]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

NameError: name 'model' is not defined

### Callbacks

In [33]:
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

NameError: name 'ReduceLROnPlateau' is not defined

### Train

In [34]:
%%time
history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

NameError: name 'model' is not defined

### Evaluate

In [35]:
%%time
score = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

NameError: name 'model' is not defined

In [36]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

NameError: name 'history' is not defined

### Predict

In [37]:
def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE

In [38]:
def predict(text, include_neutral=True):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}  

In [39]:
predict("I love the music")

NameError: name 'time' is not defined

In [40]:
predict("I hate the rain")

NameError: name 'time' is not defined

In [41]:
predict("i don't know what i'm doing")

NameError: name 'time' is not defined

### Confusion Matrix

In [42]:
%%time
y_pred_1d = []
y_test_1d = list(df_test.target)
scores = model.predict(x_test, verbose=1, batch_size=8000)
y_pred_1d = [decode_sentiment(score, include_neutral=False) for score in scores]

NameError: name 'df_test' is not defined

In [43]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [44]:
%%time

cnf_matrix = confusion_matrix(y_test_1d, y_pred_1d)
plt.figure(figsize=(12,12))
plot_confusion_matrix(cnf_matrix, classes=df_train.target.unique(), title="Confusion matrix")
plt.show()

NameError: name 'y_test_1d' is not defined

### Classification Report

In [45]:
print(classification_report(y_test_1d, y_pred_1d))

NameError: name 'y_test_1d' is not defined

### Accuracy Score

In [46]:
accuracy_score(y_test_1d, y_pred_1d)

NameError: name 'y_test_1d' is not defined

### Save model

In [47]:
model.save(KERAS_MODEL)
w2v_model.save(WORD2VEC_MODEL)
pickle.dump(tokenizer, open(TOKENIZER_MODEL, "wb"), protocol=0)
pickle.dump(encoder, open(ENCODER_MODEL, "wb"), protocol=0)

NameError: name 'model' is not defined